In [11]:
import pandas as pd

In [164]:
df=pd.read_excel('complete.xlsx', index_col=0)
CURRENT_YEAR = 2022

In [166]:
df = df.groupby(['SubjectName', 'Place', 'Time', 'Teacher'])['Date'].agg(Dates = lambda x: x).reset_index()
df['Dates'] = df['Dates'].apply(lambda x: parse_dates(x))
df = df.explode('Dates')
df[['DateStart', 'DateEnd', 'RepeatTime', 'RepeatType']] = pd.DataFrame(df.Dates.tolist(), index=df.index)
df.head()

,SubjectName,Place,Time,Teacher,Dates,DateStart,DateEnd,RepeatTime,RepeatType
0,"Автоматизация учета на предприятии, контрольна...","Таврическая улица, д. 21-23-25, лит. А,81",09:00–10:30,Иванова В. В.,"(2022-04-06 00:00:00, 2022-04-06 00:00:00, 1, )",2022-04-06,2022-04-06,1,
1,"Автоматизация учета на предприятии, контрольна...","Таврическая улица, д. 21-23-25, лит. А,81",10:45–12:15,Иванова В. В.,"(2022-05-11 00:00:00, 2022-05-11 00:00:00, 1, )",2022-05-11,2022-05-11,1,
2,"Автоматизация учета на предприятии, практическ...",С использованием информационно-коммуникационны...,09:00–10:30,Иванова В. В.,"(2022-02-16 00:00:00, 2022-02-23 00:00:00, 2, ...",2022-02-16,2022-02-23,2,weekly
3,"Автоматизация учета на предприятии, практическ...",С использованием информационно-коммуникационны...,10:45–12:15,Иванова В. В.,"(2022-02-16 00:00:00, 2022-02-23 00:00:00, 2, ...",2022-02-16,2022-02-23,2,weekly
4,"Автоматизация учета на предприятии, практическ...","Таврическая улица, д. 21-23-25, лит. А,81",09:00–10:30,Иванова В. В.,"(2022-03-02 00:00:00, 2022-03-30 00:00:00, 5, ...",2022-03-02,2022-03-30,5,weekly


In [149]:
import datetime as dt


def parse_dates(dates):
    if type(dates) == pd.core.arrays.datetimes.DatetimeArray:
        # dates = dates.reset_index(drop=True)
        return find_pattern(dates)
    else:
        return [(dates, dates, 1, '')]


def find_pattern(dates):
    output_data = []
    date_start = dates[0]
    date_end = dates[0]
    repeat_time = 1
    repeat_type = 'weekly'
    for date in dates[1:]:
        if (date_end + dt.timedelta(days=7) == date and repeat_type == 'weekly') or (
                date_end + dt.timedelta(days=14) == date and repeat_type == 'biweekly'):
            date_end = date
            repeat_time += 1
        else:
            if date_end + dt.timedelta(days=7) == date:
                if repeat_time == 1:
                    repeat_time += 1
                    date_end = date
                    repeat_type = 'weekly'
                    continue
                output_data.append((date_start, date_end, repeat_time, repeat_type))
                date_start, date_end, repeat_time, repeat_type = date, date, 1, 'weekly'
                continue
            if date_end + dt.timedelta(days=14) == date:
                if repeat_time == 1:
                    repeat_time += 1
                    date_end = date
                    repeat_type = 'biweekly'
                    continue
                output_data.append((date_start, date_end, repeat_time, repeat_type))
                date_start, date_end, repeat_time, repeat_type = date, date, 1, 'biweekly'
                continue
            if date_end == date:
                continue
            output_data.append((date_start, date_end, repeat_time, repeat_type))
            date_start, date_end, repeat_time = date, date, 1
    output_data.append((date_start, date_end, repeat_time, repeat_type))
    return output_data


def parse_time(time_str):
    def get_datetime_format(time_str: str):
        return pd.to_datetime(time_str, format='%H:%M').time()

    return list(map(get_datetime_format, time_str.split('–')))

In [170]:
df[['TimeStart', 'TimeEnd']] = pd.DataFrame(df['Time'].apply(parse_time).tolist(), index=df.index)
df = df.drop(['Dates', 'Time'], axis=1)

In [177]:
df = df[df['SubjectName'].str.match('((^(?!Траектория).*)(^(?!Русский язык).*))|(Траектория 3 .*)')]

In [178]:
df

,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime,RepeatType,TimeStart,TimeEnd
0,"Автоматизация учета на предприятии, контрольна...","Таврическая улица, д. 21-23-25, лит. А,81",Иванова В. В.,2022-04-06,2022-04-06,1,,09:00:00,10:30:00
1,"Автоматизация учета на предприятии, контрольна...","Таврическая улица, д. 21-23-25, лит. А,81",Иванова В. В.,2022-05-11,2022-05-11,1,,10:45:00,12:15:00
2,"Автоматизация учета на предприятии, практическ...",С использованием информационно-коммуникационны...,Иванова В. В.,2022-02-16,2022-02-23,2,weekly,09:00:00,10:30:00
3,"Автоматизация учета на предприятии, практическ...",С использованием информационно-коммуникационны...,Иванова В. В.,2022-02-16,2022-02-23,2,weekly,10:45:00,12:15:00
4,"Автоматизация учета на предприятии, практическ...","Таврическая улица, д. 21-23-25, лит. А,81",Иванова В. В.,2022-03-02,2022-03-30,5,weekly,09:00:00,10:30:00
4,"Автоматизация учета на предприятии, практическ...","Таврическая улица, д. 21-23-25, лит. А,81",Иванова В. В.,2022-04-13,2022-05-04,4,weekly,09:00:00,10:30:00
5,"Автоматизация учета на предприятии, практическ...","Таврическая улица, д. 21-23-25, лит. А,81",Иванова В. В.,2022-03-02,2022-05-04,10,weekly,10:45:00,12:15:00
6,"Имитационное моделирование, контрольная работа",С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2022-05-26,2022-05-26,1,,09:00:00,10:30:00
7,"Имитационное моделирование, лекция",С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2022-02-17,2022-03-31,7,weekly,09:00:00,10:30:00
8,"Имитационное моделирование, практическое занятие",С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2022-04-07,2022-05-12,6,weekly,09:00:00,10:30:00


In [522]:
df.to_csv('spring.csv')

In [183]:
from icalendar import Calendar, Event
import pytz

In [187]:
# Убран функционал подгрупп!!
class Subject:
    params = ['TimeStart', 'TimeEnd', 'DateStart', 'DateEnd', 'SubjectName', 'Place', 'Teacher', 'RepeatTime',
              'RepeatType']  #, 'SubGroup'
    tz = pytz.timezone('Europe/Moscow')

    def __init__(self, TimeStart, TimeEnd, DateStart, DateEnd, SubjectName, Place, Teacher, RepeatTime=1, RepeatType='',
                 SubGroup=None):
        self.time_start = TimeStart
        self.time_end = TimeEnd
        self.date_start = DateStart
        self.date_end = DateEnd
        self.subject_name = SubjectName if not SubGroup else SubjectName + '#' + f'{SubGroup}'
        self.place = Place
        self.teacher = Teacher
        self.repeat_time = RepeatTime
        self.repeat_type = RepeatType
        self.sub_group = SubGroup

    @property
    def date_time_start_event(self):
        return self.get_date_time(self.date_start, self.time_start)

    @property
    def date_time_end_event(self):
        return self.get_date_time(self.date_start, self.time_end)

    @classmethod
    def get_date_time(cls, date, time):
        return dt.datetime.combine(date, time, cls.tz)

    @classmethod
    def from_series(cls, series: pd.Series):
        subject_params = [series[param] for param in cls.params]
        return cls(*subject_params)

    def __repr__(self):
        return f"<Subject time start: {self.time_start} date start: {self.date_start} with {self.repeat_time} repeat>"

    def to_event(self) -> Event:
        e = Event()
        e.add('DTSTART', self.date_time_start_event)
        e.add('DTEND', self.date_time_end_event)
        e.add('SUMMARY', self.subject_name)
        e.add('DESCRIPTION', f"Преподователь: {self.teacher}")
        e.add('LOCATION', self.place)
        e.add('LAST-MODIFIED', dt.datetime.now(self.tz))
        if self.repeat_type:
            e.add('RRULE',
                  {'freq': 'weekly', 'count': self.repeat_time, 'interval': 1 if self.repeat_type == 'weekly' else 2})
        return e

In [185]:
cal = Calendar()
cal.add('prodid', '-//Google Inc//Google Calendar 70.9054//EN')

In [188]:
for idx, item in df.iterrows():
    s = Subject.from_series(item)
    e = s.to_event()
    cal.add_component(e)

In [189]:
with open('example_spring.ics', 'wb') as f:
    data = cal.to_ical()
    f.write(data)

In [190]:
df[df['SubjectName'].str.startswith('Электив')].head()

,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime,RepeatType,TimeStart,TimeEnd
57,"Электив. Визуализация данных, контрольная работа","Таврическая улица, д. 21-23-25, лит. А,81",Гадасина Л. В.,2022-04-27,2022-04-27,1,,13:15:00,14:45:00
58,"Электив. Визуализация данных, лекция","Таврическая улица, д. 21-23-25, лит. А,81",Гадасина Л. В.,2022-03-02,2022-04-20,8,weekly,13:15:00,14:45:00
59,"Электив. Визуализация данных, лекция","Таврическая улица, д. 21-23-25, лит. А,81",Гадасина Л. В.,2022-03-02,2022-04-20,8,weekly,15:00:00,16:30:00
60,"Электив. Разработка JAVA-приложений, контрольн...","Чайковского улица, д. 62, лит. А,303",Стоянова О. В.,2022-03-28,2022-03-28,1,,09:00:00,10:30:00
61,"Электив. Разработка JAVA-приложений, лекция",С использованием информационно-коммуникационны...,Стоянова О. В.,2022-02-14,2022-02-14,1,,09:00:00,10:30:00
